In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
qualifying_schema = StructType(fields=[
    StructField("qualifyId", IntegerType(), True),
    StructField("raceId", IntegerType(), True),
    StructField("driverId", IntegerType(), True),
    StructField("constructorId", IntegerType(), True),
    StructField("number", IntegerType(), True),
    StructField("position", IntegerType(), True),
    StructField("q1", StringType(), True),
    StructField("q2", StringType(), True),
    StructField("q3", StringType(), True),
])

In [0]:
qualifying_df = spark.read \
.schema(qualifying_schema) \
.csv(f"{raw_folder_path}/{v_file_date}/qualifying.csv") 



In [0]:
from pyspark.sql.functions import lit

In [0]:
qualifying_final_df = add_ingestion_date(qualifying_df) \
    .withColumnRenamed("raceId", "race_id") \
    .withColumnRenamed("qualifyId", "qualify_id") \
.withColumnRenamed("driverId", "driver_id") \
    .withColumnRenamed("constructorId", "constructor_id") \
.withColumn("data_source", lit(v_data_source))\
    .withColumn("file_date", lit(v_file_date))

In [0]:
merge_condition = "tgt.qualify_id = src.qualify_id AND tgt.race_id = src.race_id"
merge_delta_data(qualifying_final_df,'f1_processed', 'qualifying', merge_condition,'race_id')

In [0]:
dbutils.notebook.exit("success")